In [16]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama.llms import OllamaLLM
from typing import TypedDict , Annotated
from langchain_core.messages import BaseMessage , HumanMessage,SystemMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver

In [13]:
from langgraph.graph.message import add_messages
model = OllamaLLM(model="llama2",temperature=0.7)

In [14]:
class ChatState(TypedDict):
    
    message: Annotated[list[BaseMessage],add_messages]

In [15]:
def Chat_node(state:ChatState):
    
    message= state['message']
    response = model.invoke(message)
    return {'message': [response]}

In [17]:
checkpoint = MemorySaver()

graph = StateGraph(ChatState)


graph.add_node('chatNode',Chat_node)
graph.add_edge(START,'chatNode')
graph.add_edge('chatNode',END)

chatbot = graph.compile(checkpointer=checkpoint)

In [ ]:
from IPython.display import Image
Image(chatbot.get_graph().draw_mermaid_png())

In [9]:
initial_state = {'message':[HumanMessage(content="what is langgraph?")]}

chatbot.invoke(initial_state)['message'][-1].content

'Langgraph (short for language graph) is a computational model of human language that represents the structure of language as a network or graph. The model posits that the meaning of words and phrases is determined by their relationships to other words and phrases in the linguistic system, rather than by any inherent meaning they may have in isolation.\n\nIn a langgraph, words and phrases are represented as nodes, and the relationships between them are represented as edges or links. The edges may be labeled with various types of information, such as syntactic, semantic, or pragmatic relationships. For example, a sentence like "The dog chased the cat" might be represented as a langgraph with the words "dog," "chased," and "cat" as nodes, and the relationship "agent-action-patient" labeled on the edge connecting them.\n\nLanggraphs can be used to represent a wide range of linguistic phenomena, including grammatical structures, lexical semantics, and pragmatic effects. They have been appl

In [18]:
thread_id = '1'

while True:
    ques = input("Ask anything: ")
    print('user:', ques)
    if ques.strip().lower() in ['quit','exit','bye']:
        break
    config = {'configurable':{'thread_id':thread_id}}
    res = chatbot.invoke({'message':[HumanMessage(content=ques)]},config=config)
    print('AI:', res['message'][-1].content)

user: hi my name aditya
AI: Assistant: Hello Aditya! It's nice to meet you. How can I assist you today? Is there something you would like to talk about or ask?
AI: Assistant: Hello Aditya! It's nice to meet you. How can I assist you today? Is there something you would like to talk about or ask?
user: what is my name?
user: what is my name?
AI: Assistant: Hello Aditya! It's great to meet you too. By the way, your name is... *pauses* Aditya! 😊 Is there anything else on your mind that you would like to talk about or ask? I'm here to listen and help in any way I can.
AI: Assistant: Hello Aditya! It's great to meet you too. By the way, your name is... *pauses* Aditya! 😊 Is there anything else on your mind that you would like to talk about or ask? I'm here to listen and help in any way I can.
user: quit
user: quit


In [25]:
state = chatbot.get_state(config=config)
print(state)
# for i in state.values['message']:
#     print(i.content)

StateSnapshot(values={'message': [HumanMessage(content='hi my name aditya', additional_kwargs={}, response_metadata={}, id='6de8222a-acae-48ba-b63e-f1ee85e4fc41'), HumanMessage(content="Assistant: Hello Aditya! It's nice to meet you. How can I assist you today? Is there something you would like to talk about or ask?", additional_kwargs={}, response_metadata={}, id='1e6527d0-b741-4532-8209-1243808d7914'), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='cc694ba7-eb87-41d4-96b1-d8296e4d2aef'), HumanMessage(content="Assistant: Hello Aditya! It's great to meet you too. By the way, your name is... *pauses* Aditya! 😊 Is there anything else on your mind that you would like to talk about or ask? I'm here to listen and help in any way I can.", additional_kwargs={}, response_metadata={}, id='bda98fd4-f712-44f6-810b-9a604d07908f')]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d33aa-77a8-67d3-8004-b019a650b6